# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not c

#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
import emergency

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [ ]:
try:
    # OpenAI API 키가 올바르게 설정되었는지 확인
    assert emergency.openai.api_key is not None, "OpenAI API Key is not set!"
    print("API키를 정상적으로 불러왔습니다.")
except AssertionError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"error: {e}")

Error: OpenAI API Key is not set!


### (2) audio to text

In [ ]:
# Audio-to-Text 변환 테스트
audio_path = "/content/drive/MyDrive/KT/6차2/audio/"
filename = 'audio1.mp3'

print("대화를 텍스트로 바꾸는중...")
try:
    transcript = emergency.audio_to_text(audio_path, filename, emergency.openai)
    print(transcript)
except Exception as e:
    print(f"Error during audio-to-text conversion: {e}")

대화를 텍스트로 바꾸는중...
지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.



### (3) text summary

In [ ]:
print("텍스트를 요약하는중...")
try:
    summary = emergency.text_summary(transcript, emergency.openai)
    print(summary)
except Exception as e:
    print(f"Error during text summarization: {e}")

텍스트를 요약하는중...
마흔아홉 살인 아버지가 계단에서 넘어져 머리에서 피가 나고 어지럽다고 합니다. 응급실로 신속히 이동하는 것이 좋을 것 같습니다.


### (4) 응급실 등급분류

In [ ]:
save_directory = path + "fine_tuned_bert_all"

In [ ]:
print("모델 예측 실행중...")
try:
    model, tokenizer = emergency.load_model_and_tokenizer(save_directory)
    predicted_class, probabilities = emergency.predict(summary, model, tokenizer)
    print(f"예측된 클래스: {predicted_class}")
    print(f"확신정도: {probabilities}")
except Exception as e:
    print(f"Error during model prediction: {e}")

모델 예측 실행중...
예측된 클래스: 1
확신정도: [[0.1080993041396141, 0.7797561287879944, 0.08237750828266144, 0.019138062372803688, 0.010629006661474705]]


### (5) 응급실추천

In [ ]:
latitude = 37.5665
longitude = 126.9780
data = pd.read_csv('/content/drive/MyDrive/KT/6차2/emergency_medical_info.csv')

In [ ]:
print("주변 응급실 검색 및 추천 중...")
try:
    recommended_hospitals = emergency.find_emergency(
        lat=latitude,
        lon=longitude,
        data=data,
        c_id=emergency.c_id,
        c_key=emergency.c_key
    )
    print("추천 응급실:\n", recommended_hospitals)
except Exception as e:
    print(f"Error during distance calculation or recommendation: {e}")

주변 응급실 검색 및 추천 중...
추천 응급실:
         병원이름                            주소                             상세주소  \
23    강북삼성병원        서울특별시 종로구 새문안로 29 (평동)                       서대문역 4번 출구   
180  서울적십자병원  서울특별시 종로구 새문안로 9, 적십자병원 (평동)  서대문 사거리 지하철 5호선 서대문역 4번 출구 10미터   
195     세란병원       서울특별시 종로구 통일로 256 (무악동)                       독립문역 1번 출구   

             대표전화         응급실전화  응급실운영여부  병상수 응급의료기관분류명          경도  \
23   02-2001-2001  02-2001-1000        1  720  지역응급의료센터  126.967938   
180  02-2002-8000  02-2002-8888        1  292  지역응급의료기관  126.966999   
195   02-737-0181   02-737-0181        1  204  지역응급의료기관  126.957707   

            위도  도로 거리  
23   37.568498  1.373  
180  37.567155  1.836  
195  37.575340  3.031  


## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [ ]:
def emergency_pipeline(audio_path, filename, latitude, longitude, hospital_data_path, model_path):
    # 1. OpenAI API 준비
    client = emergency.openai

    # 2. 오디오 파일 -> 텍스트 변환
    print("오디오를 텍스트로 변환하는 중...")
    transcript = emergency.audio_to_text(audio_path, filename, client)
    print(transcript)

    # 3. 텍스트 요약
    print("텍스트 요약중...")
    summary = emergency.text_summary(transcript, client)
    print(summary)

    # 4. 모델과 토크나이저 로드
    print("모델 로딩 중...")
    model, tokenizer = emergency.load_model_and_tokenizer(model_path)

    # 5. 응급 등급 예측
    print("모델 예측 중...")
    predicted_class, probabilities = emergency.predict(summary, model, tokenizer)
    print(f"예측된 클래스: {predicted_class}, 확신 정도: {probabilities}")

    # 6. 응급 등급에 따른 로직
    if predicted_class in [1, 2, 3]:
        # 응급 등급 1~3: 병원 추천
        print("상처 정도가 심각합니다. 주변 응급실을 검색합니다.")

        # 6.1 병원 데이터 로드
        print("응급실 데이터 로딩...")
        hospital_data = pd.read_csv(hospital_data_path)

        # 6.2 주변 병원 추천
        print("근처 병원 3군데를 찾았습니다.")
        recommended_hospitals = emergency.find_emergency(
            lat=latitude,
            lon=longitude,
            data=hospital_data,
            c_id=emergency.c_id,
            c_key=emergency.c_key
        )

        print("찾은 응급실:\n", recommended_hospitals)
        return recommended_hospitals
    else:
        print("응급 환자가 아닙니다.")
        return "응급 환자가 아닙니다."

# 사용 예시
path = "/content/drive/MyDrive/KT/6차2/"
audio_path = path + 'audio'
filename = 'audio1.mp3'
latitude = 37.5665  # 서울 시청
longitude = 126.9780
hospital_data_path = path + "emergency_medical_info.csv"
model_path = path + "fine_tuned_bert_all"

# 파이프라인 실행
recommended_hospitals = emergency_pipeline(audio_path, filename, latitude, longitude, hospital_data_path, model_path)


오디오를 텍스트로 변환하는 중...
지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.

텍스트 요약중...
마흔아홉 살의 아버지가 계단에서 넘어져 머리에서 피가 나고 어질러하다고 하며 응급 상황일 수 있습니다. 빠른 시일 내에 응급실로 이동하여 의료진의 치료를 받는 것이 중요합니다.
모델 로딩 중...
모델 예측 중...
예측된 클래스: 1, 확신 정도: [[0.053577229380607605, 0.6740899682044983, 0.23738650977611542, 0.02065207064151764, 0.014294194057583809]]
상처 정도가 심각합니다. 주변 응급실을 검색합니다.
응급실 데이터 로딩...
근처 병원 3군데를 찾았습니다.
찾은 응급실:
         병원이름                            주소                             상세주소  \
23    강북삼성병원        서울특별시 종로구 새문안로 29 (평동)                       서대문역 4번 출구   
180  서울적십자병원  서울특별시 종로구 새문안로 9, 적십자병원 (평동)  서대문 사거리 지하철 5호선 서대문역 4번 출구 10미터   
195     세란병원       서울특별시 종로구 통일로 256 (무악동)                       독립문역 1번 출구   

             대표전화         응급실전화  응급실운영여부  병상수 응급의료기관분류명          경도  \
23   02-2001-2001  02-2001-1000        1  720  지역응급의료센터  126.967938   
180  02-2002-8000  02-2002-8888

In [ ]:
recommended_hospitals

,병원이름,주소,상세주소,대표전화,응급실전화,응급실운영여부,병상수,응급의료기관분류명,경도,위도,도로 거리
23,강북삼성병원,서울특별시 종로구 새문안로 29 (평동),서대문역 4번 출구,02-2001-2001,02-2001-1000,1,720,지역응급의료센터,126.967938,37.568498,1.309
180,서울적십자병원,"서울특별시 종로구 새문안로 9, 적십자병원 (평동)",서대문 사거리 지하철 5호선 서대문역 4번 출구 10미터,02-2002-8000,02-2002-8888,1,292,지역응급의료기관,126.966999,37.567155,1.772
195,세란병원,서울특별시 종로구 통일로 256 (무악동),독립문역 1번 출구,02-737-0181,02-737-0181,1,204,지역응급의료기관,126.957707,37.575340,3.031
